# Inference + Intervals


Load model bundles and generate 24h forecasts with intervals.

In [3]:
import sys, subprocess
from pathlib import Path

print("Python:", sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
subprocess.run(["pip", "install", "-e", str(repo_root)], check=True)


Python: /Users/pratik_n/Downloads/gridpulse/.venv/bin/python
Obtaining file:///Users/pratik_n/Downloads/gridpulse
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for gridpulse (pyproject.toml): started
  Building editable for gridpulse (pyproject.toml): finished with status 'done'
  Created wheel for gridpulse: filename=gridpulse-0.1.0-0.editable-py3-none-any.whl size=6066 sha256=4b7016a440b6e9d1fd9668cd39a3e492c11db436c62719bbeda28cb8bc842fad
  Stored in directory: /private/var/folders/pk/ksqc41pj60q3589gfwhqz5tw0000gn/T/pip-

CompletedProcess(args=['pip', 'install', '-e', '/Users/pratik_n/Downloads/gridpulse'], returncode=0)

In [4]:
import pandas as pd
from pathlib import Path
from gridpulse.forecasting.predict import load_model_bundle, predict_next_24h

features_path = Path(repo_root) / "data" / "processed" / "features.parquet"
model_path = Path(repo_root) / "artifacts" / "models" / "lstm_load_mw.pt"

print("Features:", features_path)
print("Model:", model_path)

features = pd.read_parquet(features_path)
bundle = load_model_bundle(model_path)
forecast = predict_next_24h(features, bundle)
forecast


Features: /Users/pratik_n/Downloads/gridpulse/data/processed/features.parquet
Model: /Users/pratik_n/Downloads/gridpulse/artifacts/models/lstm_load_mw.pt


{'target': 'load_mw',
 'model': 'lstm',
 'timestamp': ['2020-10-01T00:00:00+00:00',
  '2020-10-01T01:00:00+00:00',
  '2020-10-01T02:00:00+00:00',
  '2020-10-01T03:00:00+00:00',
  '2020-10-01T04:00:00+00:00',
  '2020-10-01T05:00:00+00:00',
  '2020-10-01T06:00:00+00:00',
  '2020-10-01T07:00:00+00:00',
  '2020-10-01T08:00:00+00:00',
  '2020-10-01T09:00:00+00:00',
  '2020-10-01T10:00:00+00:00',
  '2020-10-01T11:00:00+00:00',
  '2020-10-01T12:00:00+00:00',
  '2020-10-01T13:00:00+00:00',
  '2020-10-01T14:00:00+00:00',
  '2020-10-01T15:00:00+00:00',
  '2020-10-01T16:00:00+00:00',
  '2020-10-01T17:00:00+00:00',
  '2020-10-01T18:00:00+00:00',
  '2020-10-01T19:00:00+00:00',
  '2020-10-01T20:00:00+00:00',
  '2020-10-01T21:00:00+00:00',
  '2020-10-01T22:00:00+00:00',
  '2020-10-01T23:00:00+00:00'],
 'forecast': [178.3181610107422,
  177.99319458007812,
  178.3347625732422,
  178.2835235595703,
  178.2949981689453,
  178.5199432373047,
  178.8855438232422,
  177.95736694335938,
  177.82479858398438

In [5]:
import pandas as pd
import plotly.express as px

rows = pd.DataFrame({
    "timestamp": forecast["timestamp"],
    "p50": forecast["forecast"],
})
for q, vals in forecast.get("quantiles", {}).items():
    rows[f"q{q}"] = vals

rows["timestamp"] = pd.to_datetime(rows["timestamp"], utc=True)
fig = px.line(rows, x="timestamp", y=[c for c in rows.columns if c != "timestamp"])
fig



In [6]:
import plotly.express as px

fig = px.line(
    rows,
    x="timestamp",
    y=[c for c in rows.columns if c != "timestamp"],
    title="Forecast + Quantile Bands"
)
fig


In [7]:
import pandas as pd
import plotly.express as px

tmp = rows.copy()
tmp["hour"] = pd.to_datetime(tmp["timestamp"]).dt.hour
hourly = tmp.groupby("hour")["p50"].mean().reset_index()

fig = px.bar(hourly, x="hour", y="p50", title="Avg Forecast by Hour")
fig


In [8]:
import plotly.express as px

fig = px.histogram(rows, x="p50", nbins=40, title="Forecast Distribution")
fig


In [9]:
import pandas as pd
import plotly.express as px

tmp = rows.copy()
tmp["hour"] = pd.to_datetime(tmp["timestamp"]).dt.hour
tmp["bucket"] = pd.cut(tmp["hour"], bins=[0,6,12,18,24], labels=["Night","Morning","Afternoon","Evening"], right=False)

fig = px.box(tmp, x="bucket", y="p50", title="Forecast Spread by Time of Day")
fig


In [10]:
import pandas as pd
import plotly.express as px

tmp = rows.copy()
tmp["hour"] = pd.to_datetime(tmp["timestamp"]).dt.hour

heat = tmp.groupby("hour")[["p50"]].mean().reset_index()
fig = px.imshow(
    heat.set_index("hour").T,
    aspect="auto",
    title="Heatmap: Avg Forecast by Hour"
)
fig
